# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [56]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import gmaps

# Google developer API key
from pprint import pprint

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [57]:
weather_file = "../output_data/cities.csv"
# output_data_file

weather_df = pd.read_csv(weather_file)

weatherpy_df = weather_df.drop(columns=["Unnamed: 0"])
weatherpy_df

,City,Country,Date,Lat,Lng,Humidity,Cloudiness,Wind Speed,Max Temp
0,taolanaro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,hermanus,ZA,2020-04-11 16:46:54,-34.42,19.23,93.0,100.0,18.01,63.00
2,atuona,PF,2020-04-11 16:46:55,-9.80,-139.03,66.0,48.0,5.77,82.89
3,margate,GB,2020-04-11 16:46:55,51.38,1.39,87.0,41.0,3.36,57.00
4,mataura,NZ,2020-04-11 16:46:55,-46.19,168.86,58.0,93.0,1.99,64.00
...,...,...,...,...,...,...,...,...,...
601,korla,CN,2020-04-11 16:49:40,41.76,86.15,41.0,69.0,4.00,59.00
602,kuty,UA,2020-04-11 16:49:41,48.26,25.18,67.0,0.0,1.21,37.54
603,mafeteng,LS,2020-04-11 16:49:41,-29.82,27.24,38.0,4.0,7.54,56.34
604,marrakesh,MA,2020-04-11 16:49:41,31.63,-8.01,72.0,94.0,5.82,60.80


In [58]:
dataTypeSeries = weatherpy_df.dtypes
dataTypeSeries

City           object
Country        object
Date           object
Lat           float64
Lng           float64
Humidity      float64
Cloudiness    float64
Wind Speed    float64
Max Temp      float64
dtype: object

In [59]:
weatherpy_df = weatherpy_df.dropna(how="any")
weatherpy_df

,City,Country,Date,Lat,Lng,Humidity,Cloudiness,Wind Speed,Max Temp
1,hermanus,ZA,2020-04-11 16:46:54,-34.42,19.23,93.0,100.0,18.01,63.00
2,atuona,PF,2020-04-11 16:46:55,-9.80,-139.03,66.0,48.0,5.77,82.89
3,margate,GB,2020-04-11 16:46:55,51.38,1.39,87.0,41.0,3.36,57.00
4,mataura,NZ,2020-04-11 16:46:55,-46.19,168.86,58.0,93.0,1.99,64.00
5,vila franca do campo,PT,2020-04-11 16:46:55,37.72,-25.43,82.0,20.0,12.75,57.20
...,...,...,...,...,...,...,...,...,...
601,korla,CN,2020-04-11 16:49:40,41.76,86.15,41.0,69.0,4.00,59.00
602,kuty,UA,2020-04-11 16:49:41,48.26,25.18,67.0,0.0,1.21,37.54
603,mafeteng,LS,2020-04-11 16:49:41,-29.82,27.24,38.0,4.0,7.54,56.34
604,marrakesh,MA,2020-04-11 16:49:41,31.63,-8.01,72.0,94.0,5.82,60.80


In [60]:
dataTypeSeries = weatherpy_df.dtypes
dataTypeSeries

City           object
Country        object
Date           object
Lat           float64
Lng           float64
Humidity      float64
Cloudiness    float64
Wind Speed    float64
Max Temp      float64
dtype: object

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [61]:
#configure gkey
gmaps.configure(api_key=g_key)

#lat and lng coordinates
coordinates = weatherpy_df[["Lat", "Lng"]]

#humidity
humidity = weatherpy_df["Humidity"].astype(float)

In [62]:

# Customize the size of the figure
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

In [63]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [64]:
ideal_temp_df = weatherpy_df.loc[pd.to_numeric(weatherpy_df["Max Temp"]).astype(float).between(74, 78, inclusive=False)]

In [65]:
ideal_temp_wind_df = ideal_temp_df.loc[pd.to_numeric(ideal_temp_df["Wind Speed"]).astype(float) <10]

In [66]:
ideal_temp_wind_cloud_df = ideal_temp_wind_df.loc[pd.to_numeric(ideal_temp_wind_df["Cloudiness"]).astype(float) <50]

In [67]:
ideal_vacay_df = ideal_temp_wind_df.loc[pd.to_numeric(ideal_temp_wind_df["Humidity"]).astype(float) <70]
ideal_vacay_df

,City,Country,Date,Lat,Lng,Humidity,Cloudiness,Wind Speed,Max Temp
100,chicama,PE,2020-04-11 16:43:10,-7.84,-79.15,67.0,63.0,7.67,76.19
124,port hedland,AU,2020-04-11 16:47:32,-20.32,118.57,64.0,5.0,6.93,75.20
170,bilma,NE,2020-04-11 16:43:19,18.69,12.92,16.0,100.0,5.01,77.07
171,marawi,PH,2020-04-11 16:47:47,8.00,124.29,56.0,54.0,1.95,77.27
187,praia,CV,2020-04-11 16:47:51,14.92,-23.51,69.0,82.0,9.17,75.20
241,yulara,AU,2020-04-11 16:48:05,-25.24,130.99,27.0,100.0,8.05,75.20
295,chitapur,IN,2020-04-11 16:48:19,17.12,77.08,31.0,0.0,3.76,77.65
363,biu,NG,2020-04-11 16:48:35,10.61,12.19,14.0,97.0,7.25,76.24
365,puerto maldonado,PE,2020-04-11 16:48:36,-12.60,-69.18,61.0,0.0,1.57,74.53
392,barhiya,IN,2020-04-11 16:48:44,25.28,86.03,20.0,5.0,0.72,76.19


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [68]:
hotel_df = pd.DataFrame(ideal_vacay_df, columns=["City", "Country", "Lat", "Lng", "Humidity", "Cloudiness", "Wind Speed", "Max Temp"])
hotel_df

,City,Country,Lat,Lng,Humidity,Cloudiness,Wind Speed,Max Temp
100,chicama,PE,-7.84,-79.15,67.0,63.0,7.67,76.19
124,port hedland,AU,-20.32,118.57,64.0,5.0,6.93,75.20
170,bilma,NE,18.69,12.92,16.0,100.0,5.01,77.07
171,marawi,PH,8.00,124.29,56.0,54.0,1.95,77.27
187,praia,CV,14.92,-23.51,69.0,82.0,9.17,75.20
241,yulara,AU,-25.24,130.99,27.0,100.0,8.05,75.20
295,chitapur,IN,17.12,77.08,31.0,0.0,3.76,77.65
363,biu,NG,10.61,12.19,14.0,97.0,7.25,76.24
365,puerto maldonado,PE,-12.60,-69.18,61.0,0.0,1.57,74.53
392,barhiya,IN,25.28,86.03,20.0,5.0,0.72,76.19


In [69]:
hotel_df['Hotel Name'] = ""
hotel_df

,City,Country,Lat,Lng,Humidity,Cloudiness,Wind Speed,Max Temp,Hotel Name
100,chicama,PE,-7.84,-79.15,67.0,63.0,7.67,76.19,
124,port hedland,AU,-20.32,118.57,64.0,5.0,6.93,75.20,
170,bilma,NE,18.69,12.92,16.0,100.0,5.01,77.07,
171,marawi,PH,8.00,124.29,56.0,54.0,1.95,77.27,
187,praia,CV,14.92,-23.51,69.0,82.0,9.17,75.20,
241,yulara,AU,-25.24,130.99,27.0,100.0,8.05,75.20,
295,chitapur,IN,17.12,77.08,31.0,0.0,3.76,77.65,
363,biu,NG,10.61,12.19,14.0,97.0,7.25,76.24,
365,puerto maldonado,PE,-12.60,-69.18,61.0,0.0,1.57,74.53,
392,barhiya,IN,25.28,86.03,20.0,5.0,0.72,76.19,


In [70]:
params = { 
    "radius": 5000,
    "type": "hotel",
    "key": g_key,
}
# Use the lat/lng to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    # get city from df
    city_name = row['City']

    # change location each iteration while leaving original params in place(param from google)
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make request and print url
    hotel_name = requests.get(base_url, params=params)

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    #pprint(response)

    # extract results
    results = response
    #pprint(results)
    
    try:
        
        hotel_df.loc[index, 'Hotel Name'] = results['results'][1]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 100: chicama.
------------
Retrieving Results for Index 124: port hedland.
------------
Retrieving Results for Index 170: bilma.
------------
Retrieving Results for Index 171: marawi.
------------
Retrieving Results for Index 187: praia.
------------
Retrieving Results for Index 241: yulara.
------------
Retrieving Results for Index 295: chitapur.
------------
Retrieving Results for Index 363: biu.
------------
Retrieving Results for Index 365: puerto maldonado.
------------
Retrieving Results for Index 392: barhiya.
------------
Retrieving Results for Index 418: balugaon.
------------
Retrieving Results for Index 443: yaan.
------------
Retrieving Results for Index 477: lahij.
Missing field/result... skipping.
------------
Retrieving Results for Index 491: mount isa.
------------
Retrieving Results for Index 535: acari.
------------
Retrieving Results for Index 577: pignon.
------------


In [71]:
hotel_df

,City,Country,Lat,Lng,Humidity,Cloudiness,Wind Speed,Max Temp,Hotel Name
100,chicama,PE,-7.84,-79.15,67.0,63.0,7.67,76.19,Balihai Bungalows
124,port hedland,AU,-20.32,118.57,64.0,5.0,6.93,75.20,The Esplanade Hotel
170,bilma,NE,18.69,12.92,16.0,100.0,5.01,77.07,Централна джамия Билма
171,marawi,PH,8.00,124.29,56.0,54.0,1.95,77.27,Marawi City Hall
187,praia,CV,14.92,-23.51,69.0,82.0,9.17,75.20,LT Aparthotel
241,yulara,AU,-25.24,130.99,27.0,100.0,8.05,75.20,Desert Gardens Hotel - Ayers Rock Resort
295,chitapur,IN,17.12,77.08,31.0,0.0,3.76,77.65,SM Money Plant
363,biu,NG,10.61,12.19,14.0,97.0,7.25,76.24,M.I USMAN COMPUTER BUSINESS CENTRE
365,puerto maldonado,PE,-12.60,-69.18,61.0,0.0,1.57,74.53,Wasai Hotel
392,barhiya,IN,25.28,86.03,20.0,5.0,0.72,76.19,"Life Insurance agent in Barahiya, Bihar"


In [72]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [73]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig
# Display Map

Figure(layout=FigureLayout(height='420px'))